In [18]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tkinter import *
from PIL import Image, ImageTk 

In [19]:
def macro_soft_f1(y, y_hat):
    """Compute the macro soft F1-score as a cost (average 1 - soft-F1 across all labels).
    Use probability values instead of binary predictions.

    Args:
        y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)

    Returns:
        cost (scalar Tensor): value of the cost function for the batch
    """
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
    macro_cost = tf.reduce_mean(cost) # average on all labels
    return macro_cost


@tf.function
def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)

    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive

    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [20]:

# Load the pre-trained model
model = load_model('dodeLidoModel', custom_objects={'macro_soft_f1': macro_soft_f1, 'macro_f1': macro_f1})

# Define class names
class_names = ["Alarm", "Blue", "Flamingo", "Giraffe", "Green", "Grey", "Lion", "Monkey", "Pink", "Sloth", "Snake", "Yellow"]


In [ ]:
X,y=val_ds1
images=X[0]
labels=np.array(X[1])

In [29]:
predictions=np.array(model.predict(images))
[predictions[0]]

KeyError: 'pop from an empty set'

In [28]:
# Define the threshold
threshold = 0.5

# Convert values to binary array based on the threshold
binary_array = np.where(np.array(predictions) > threshold, 1, 0)

NameError: name 'predictions' is not defined

In [ ]:
num_classes = len(mlb.classes_)  # Get the number of classes from mlb

#actual_labels = tf.one_hot(labels, num_classes)  # One-hot encode labels

In [ ]:
predicted_labels=mlb.inverse_transform(binary_array)
actual_labels=mlb.inverse_transform(labels)

for i in range(len(predicted_labels)):
  print(actual_labels[i],predicted_labels[i])

In [26]:
IMG_SIZE = 224
def read_image(image_object):
  image_resized = tf.image.resize(image_object,[IMG_SIZE, IMG_SIZE])
  # Normalize it from [0, 255] to [0.0, 1.0]
  image_normalized = image_resized / 255.0
  expanded_tensor = tf.expand_dims(image_normalized, axis=0)
  ouptut=model.predict(expanded_tensor)
  result_list = list(ouptut)
  binary_array = np.where(np.array(result_list) > threshold, 1, 0)
  result=mlb.inverse_transform(binary_array)
  return result


def filter_overlapping_circles(circles, min_dist_between_circles):
  filtered_circles = []

  for circle in circles[0]:
      x, y, r = circle
      overlap = False
      for fc in filtered_circles:
          fx, fy, fr = fc
          distance = np.sqrt((x - fx) ** 2 + (y - fy) ** 2)
          if distance < min_dist_between_circles:
              overlap = True
              break
      if not overlap:
          filtered_circles.append(circle)

  return (filtered_circles)

In [27]:

path="IMG_2018.JPG"
target_width = 1024
target_height = 1024
image=cv2.imread(path)
# image = cv2.resize(cv2_image, (target_width, target_height))


# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# Use Hough Circle Transform to detect circles
blurred = cv2.GaussianBlur(gray, (11, 11), 0)
circles = cv2.HoughCircles(
    blurred,
    cv2.HOUGH_GRADIENT,
    dp=1,
    minDist=500,         # Adjust based on expected circle density
    param1=100,         # Higher threshold for Canny edge detector
    param2=30,          # Lower accumulator threshold for circle detection
    minRadius=100,       # Adjust based on the smallest expected circle size
    maxRadius=450       # Adjust based on the largest expected circle size
)

images=[]
buffer=20
if circles is not None:
    # Convert the (x, y) coordinates and radius of the circles to integers
    circles = circles.astype(int)
    min_dist_between_circles = 500  # Adjust this value as needed
    filtered_circles = filter_overlapping_circles(circles, min_dist_between_circles)

    # Loop over all detected circles
    for circle in filtered_circles:
        # Extract the coordinates and radius of the circle
        x, y, r = circle

        # Draw the bounding box around the circle
        x1 = max(0, x - r-buffer)
        y1 = max(0, y - r-buffer)
        x2 = min(image.shape[1], x + r+buffer)
        y2 = min(image.shape[0], y + r+buffer)

        new_image= image[y1:y2, x1:x2]
        temp=read_image(cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB))
        print(temp)
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        text_to_display = temp[0][0] + ", " + temp[0][1]
        cv2.putText(image, text_to_display, (x1, y1- 20), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 0, 0), 2)

        images.append(image)
        # Draw the bounding box on the original image



1/1 [==============================] - 10s 10s/step


NameError: name 'mlb' is not defined

In [ ]:
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))